In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from IPython.display import display, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import functions as funcs
import cell_class as cell


In [ ]:
# Change output figure size
# ...needs to be in its own cell for some reason...

plt.rcParams['figure.figsize'] = [20, 5]

Jia, H., Rochefort, N. L., Chen, X., & Konnerth, A. (2011).
In vivo two-photon imaging of sensory-evoked dendritic calcium signals in cortical neurons.
Nature protocols, 6(1), 28.

Podgorski, K., & Haas, K. (2013).
Fast non‐negative temporal deconvolution for laser scanning microscopy.
Journal of biophotonics, 6(2), 153-162.

Coleman, P. (2019).
pyNeuroTrace. https://github.com/padster/pyNeuroTrace.git



To install pyneuortrace use this:
 pip install --upgrade "git+https://github.com/padster/pyNeuroTrace#egg=pyneurotrace&subdirectory=pyneurotrace"

## Global Settings

In [ ]:
DATA_DIRECTORY = '/mnt/5404b8a5-71b7-4464-9a1e-b40cd26fac58/Data_Drive/Wissam/osfstorage-archive/Data (Images)/'

# Framerate of video in hz
HZ = 10

# Standard Deviation Threshold Multiplier
STD_THRESH = 5

# Power for Butterworth Lowpass filter
Power = 3

# Frequency cut-off
FC = 3

### Collects all the cells in the analysis data directory and groups them by condition in two lists

In [ ]:
path = os.fspath(DATA_DIRECTORY)
cells = sorted((os.listdir(path)))
WT_Cells = []
YAC128_Cells = []
for folder in cells:
    if 'WT'  in folder:
        WT_Cells.append(folder[:-2])
    if 'YAC128'  in folder:
        YAC128_Cells.append(folder[:-2])
        
WT_Cells = np.unique(WT_Cells)
YAC128_Cells = np.unique(YAC128_Cells)


In [ ]:
# Preform analysis

wt_results = pd.DataFrame(columns=['Name', 'Cell'])
for exp in tqdm(WT_Cells):
    wt_results = wt_results.append({'Name':exp,
                       'Cell': cell.cell(DATA_DIRECTORY, exp, HZ, Power, FC, STD_THRESH)}, ignore_index=True )
    

In [ ]:
# Preform analysis

yac128_results = pd.DataFrame(columns=['Name', 'Cell'])
for exp in tqdm(YAC128_Cells):
    yac128_results = yac128_results.append({'Name':exp,
                       'Cell': cell.cell(DATA_DIRECTORY, exp, HZ, Power, FC, STD_THRESH)}, ignore_index=True )
    

In [ ]:
list1 = []
list1.append(3)

In [ ]:
list1

In [ ]:
WT_Stats = pd.DataFrame()
WT_Shared_Peaks = pd.DataFrame()
WT_GCaMP = pd.DataFrame()
WT_RCaMP = pd.DataFrame()

WT_GTrace = pd.DataFrame()
WT_RTrace = pd.DataFrame()

WT_GDFF = []
WT_RDFF = []

WT_GDFF_index = []
WT_RDFF_index = []

for i in range(wt_results.shape[0]):
    WT_GDFF_index.append(wt_results.iloc[i,1].cell)
    WT_RDFF_index.append(wt_results.iloc[i,1].cell)
    
    WT_RDFF.append(pd.DataFrame(wt_results.iloc[i,1].nndR))
    WT_GDFF.append(pd.DataFrame(wt_results.iloc[i,1].nndG))
        
    WT_Stats = WT_Stats.append(wt_results.iloc[i,1].cell_stats, ignore_index=True)
    WT_Shared_Peaks = WT_Shared_Peaks.append(wt_results.iloc[i,1].shared_peak_data, ignore_index=True)
    WT_GCaMP = WT_GCaMP.append(wt_results.iloc[i,1].gcamp_peak_data, ignore_index=True)
    WT_RCaMP = WT_RCaMP.append(wt_results.iloc[i,1].rcamp_peak_data, ignore_index=True)

WT_GDFF = pd.concat(WT_GDFF, axis=1) 
WT_GDFF = WT_GDFF.transpose()
WT_GDFF.insert(loc=0, column='Cell', value=WT_GDFF_index)
WT_RDFF = pd.concat(WT_RDFF, axis=1)   
WT_RDFF = WT_RDFF.transpose()
WT_RDFF.insert(loc=0, column='Cell', value=WT_RDFF_index)

## Save Results for WT to CSV

In [ ]:
np.set_printoptions(threshold=8000)
WT_Stats.to_csv("WT_Stats.csv", index=False)
WT_Shared_Peaks.to_csv("WT_Shared_Peaks.csv", index=False)
WT_GCaMP.to_csv("WT_GCaMP.csv", index=False)
WT_RCaMP.to_csv("WT_RCaMP.csv", index=False)

WT_RTrace.to_csv("WT_R_Trace.csv", index=False)
WT_GTrace.to_csv("WT_G_Trace.csv", index=False)
WT_RDFF.to_csv("WT_R_DFF.csv", index=False)
WT_GDFF.to_csv("WT_G_DFF.csv", index=False)

np.set_printoptions(threshold=10)

## Display Results for WT

In [ ]:
display(WT_Stats)
display(WT_Shared_Peaks)
display(WT_GCaMP)
display(WT_RCaMP)

## Inspect WT Results

In [ ]:
def f(Experiment):
    data= wt_results.index[wt_results['Name']==Experiment][0]
    wt_results['Cell'][data].inspect_peaks()
interact(f, Experiment=wt_results['Name'])

In [ ]:
def f(Experiment):
    data= wt_results.index[wt_results['Name']==Experiment][0]
    wt_results['Cell'][data].inspect_results()
interact(f, Experiment=wt_results['Name'])

## Cycle Through YAC128 Cells to Extract Peak Data


In [ ]:
YAC128_Stats = pd.DataFrame()
YAC128_Shared_Peaks = pd.DataFrame()
YAC128_GCaMP = pd.DataFrame()
YAC128_RCaMP = pd.DataFrame()

YAC128_GTrace = pd.DataFrame()
YAC128_RTrace = pd.DataFrame()

YAC128_GDFF = []
YAC128_RDFF = []

YAC128_GDFF_index = []
YAC128_RDFF_index = []

for i in range(yac128_results.shape[0]):
    YAC128_GDFF_index.append(yac128_results.iloc[i,1].cell)
    YAC128_RDFF_index.append(yac128_results.iloc[i,1].cell)
    
    YAC128_RDFF.append(pd.DataFrame(yac128_results.iloc[i,1].nndR))
    YAC128_GDFF.append(pd.DataFrame(yac128_results.iloc[i,1].nndG))
    
    YAC128_RTrace = YAC128_RTrace.append(pd.Series([yac128_results.iloc[i,1].cell, yac128_results.iloc[i,1].nndR], name=yac128_results.iloc[i,1].cell))
    YAC128_GTrace = YAC128_GTrace.append(pd.Series([yac128_results.iloc[i,1].cell, yac128_results.iloc[i,1].nndG], name=yac128_results.iloc[i,1].cell)) 

    
    YAC128_Stats = YAC128_Stats.append(yac128_results.iloc[i,1].cell_stats, ignore_index=True)
    YAC128_Shared_Peaks = YAC128_Shared_Peaks.append(yac128_results.iloc[i,1].shared_peak_data, ignore_index=True)
    YAC128_GCaMP = YAC128_GCaMP.append(yac128_results.iloc[i,1].gcamp_peak_data, ignore_index=True)
    YAC128_RCaMP = YAC128_RCaMP.append(yac128_results.iloc[i,1].rcamp_peak_data, ignore_index=True)

YAC128_GDFF = pd.concat(YAC128_GDFF, axis=1) 
YAC128_GDFF = YAC128_GDFF.transpose()
YAC128_GDFF.insert(loc=0, column='Cell', value=YAC128_GDFF_index)
YAC128_RDFF = pd.concat(YAC128_RDFF, axis=1)   
YAC128_RDFF = YAC128_RDFF.transpose()
YAC128_RDFF.insert(loc=0, column='Cell', value=YAC128_RDFF_index)

## Save Results for YAC128 to CSV

In [ ]:
np.set_printoptions(threshold=8000)
YAC128_Stats.to_csv("YAC128_Stats.csv", index=False)
YAC128_Shared_Peaks.to_csv("YAC128_Shared_Peaks.csv", index=False)
YAC128_GCaMP.to_csv("YAC128_GCaMP.csv", index=False)
YAC128_RCaMP.to_csv("YAC128_RCaMP.csv", index=False)

YAC128_RTrace.to_csv("YAC128_R_Trace.csv", index=False)
YAC128_GTrace.to_csv("YAC128_G_Trace.csv", index=False)
YAC128_RDFF.to_csv("YAC128_R_DFF.csv", index=False)
YAC128_GDFF.to_csv("YAC128_G_DFF.csv", index=False)

np.set_printoptions(threshold=10)

## Display Results for YAC128

In [ ]:
display(YAC128_Stats)
display(YAC128_Shared_Peaks)
display(YAC128_GCaMP)
display(YAC128_RCaMP)


## Inspect YAC128 Results

In [ ]:
def f(Experiment):
    data= yac128_results.index[yac128_results['Name']==Experiment][0]
    yac128_results['Cell'][data].inspect_peaks()
interact(f, Experiment=yac128_results['Name'])

In [ ]:
def f(Experiment):
    data= yac128_results.index[yac128_results['Name']==Experiment][0]
    yac128_results['Cell'][data].inspect_results()
interact(f, Experiment=yac128_results['Name'])